In [1]:
!pip install langchain pinecone replicate transformers langchain_community
!pip install pinecone chroma

In [ ]:
import os
import pinecone
import sys
from langchain.llms import Replicate
from langchain.vectorstores import Pinecone
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders.csv_loader import CSVLoader

# Set up Replicate API token
os.environ['REPLICATE_API_TOKEN'] = "YOUR_API_HERE"

# Initialize Pinecone with API key and environment
pinecone.init(api_key='YOUR_API_HERE', environment='YOUR_ENVIRONMENT_HERE')

# Load documents from CSV file
loader = CSVLoader(file_path="./somefolder/YOURFILEHERE.csv")
documents = loader.load()

# Split loaded documents into smaller chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

# Create HuggingFace embeddings
embeddings = HuggingFaceEmbeddings()

# Set up Pinecone index for storing vectors
index_name = "YOUR_INDEX_HERE"
index = pinecone.Index(index_name)
vectordb = Pinecone.from_documents(texts, embeddings, index_name=index_name)

# Initialize Replicate with LLaMA3 model
llm = Replicate(
    model="a16z-infra/llama13b-v2-chat:df7690f1994d94e96ad9d568eac121aecf50684a0b0963b25a41cc40061269e5",
    input={"temperature": 0.75, "max_length": 3000}
)

# Create a QA chain using LLaMA3 and Pinecone
qa_chain = ConversationalRetrievalChain.from_llm(
    llm,
    vectordb.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True
)

# Interact with chatbot
chat_history = []
while True:
    query = input('Prompt: ')
    if query == "exit" or query == "quit" or query == "q":
        print('Exiting')
        sys.exit()
    result = qa_chain({'question': query, 'chat_history': chat_history})
    print('Answer: ' + result['answer'] + '\n')
    chat_history.append((query, result['answer']))

In [3]:
import os
import pinecone
import sys
from langchain.llms import Replicate
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders.csv_loader import CSVLoader

# Load documents from CSV file
loader = CSVLoader(file_path="./transcript/fine_food_reviews.csv")
documents = loader.load()

# Split loaded documents into smaller chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

# Create HuggingFace embeddings
embeddings = HuggingFaceEmbeddings()

# Create a Pinecone index
pinecone.init(api_key="YOUR_API_KEY", environment="YOUR_ENVIRONMENT")
index = pinecone.Index(name="my_index", dimension=128)

# Index the documents
for text in texts:
    index.upsert([(text, embeddings.encode(text))])

# Initialize Replicate with LLaMA3 model
llm = Replicate(
    model="a16z-infra/llama13b-v2-chat:df7690f1994d94e96ad9d568eac121aecf50684a0b0963b25a41cc40061269e5",
    input={"temperature": 0.75, "max_length": 3000}
)

# Create a QA chain using LLaMA3 and the Pinecone index
def retrieve(query):
    query_embedding = embeddings.encode(query)
    results = index.query(query_embedding, top_k=2)
    return [(result.id, result.score) for result in results]

qa_chain = ConversationalRetrievalChain.from_llm(llm, retrieve, return_source_documents=True)

# Interact with chatbot
chat_history = []
while True:
    query = input('Prompt: ')
    if query == "exit" or query == "quit" or query == "q":
        print('Exiting')
        sys.exit()
    result = qa_chain({'question': query, 'chat_history': chat_history})
    print('Answer:', result['answer'] + '\n')
    chat_history.append((query, result['answer']))

/home/alysson/workspace/rag-with-chromadb-llama-index-ollama-csv/.venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

ChunkedEncodingError: ('Connection broken: IncompleteRead(355318598 bytes read, 82653274 more expected)', IncompleteRead(355318598 bytes read, 82653274 more expected))